### 한강대교 유량 결측치 처리
1. 사용할 컬럼: 자기 자신 -> 청담대교 수위 -> 행주대교 유량 -> 잠수교 수위 -> 청담대교 유량 -> 광진교 유량 -> 한강대교 수위 -> 강화대교 조위 -> 방류량 -> 강수량... -> 행주대교 수위
2. 그런데 이 순서는 모델의 행동을 설명하는 것이지 인과관계를 반드시 보장한다고 할 수는 없음...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

import time
from glob import glob
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import backend as K
# from keras.utils import plot_model

from keras.layers import Activation, Dense, LSTM, GRU, Bidirectional, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Reshape, RepeatVector, Concatenate, GlobalMaxPooling1D 
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D


#GPU 자원이 부족
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import os
os.chdir('/content/drive/MyDrive/신입기수 프로젝트/Dataset/')

In [ ]:
raw_data = pd.read_csv('rawdata.csv', encoding = 'UTF-8')

In [ ]:
raw_data

,날짜,현재수위,유입량,저수량,공용량,방류량,강화대교_조위,청담대교_유량,잠수교_유량,한강대교_유량,행주대교_유량,대곡교_강수량,진관교_강수량,송정동_강수량,청담대교_수위,잠수교_수위,한강대교_수위,행주대교_수위
0,2012-05-01 00:00,24.800,555.00,219.07,24.93,555.00,445.0,469.05,0.0,729.80,540.18,0.0,0.0,0.0,310.7,300.2,290.0,275.3
1,2012-05-01 00:10,24.794,464.60,218.86,25.15,562.90,449.0,498.00,0.0,731.48,540.18,0.0,0.0,0.0,314.7,300.2,290.0,275.3
2,2012-05-01 00:20,24.789,478.10,218.69,25.31,576.40,451.0,490.68,0.0,726.42,540.18,0.0,0.0,0.0,313.7,301.2,290.0,275.3
3,2012-05-01 00:30,24.789,464.80,218.69,25.31,563.10,452.0,476.21,0.0,726.42,552.17,0.0,0.0,0.0,311.7,301.2,290.0,276.3
4,2012-05-01 00:40,24.789,478.10,218.69,25.31,576.40,450.0,476.21,0.0,707.17,564.29,0.0,0.0,0.0,311.7,301.2,291.0,277.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2022-07-18 23:10,25.040,259.23,212.86,31.14,259.23,510.0,319.84,NaN,-456.41,974.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276332,2022-07-18 23:20,25.040,260.46,212.86,31.14,260.46,492.0,314.01,NaN,-717.30,1006.88,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276333,2022-07-18 23:30,25.040,259.37,212.86,31.14,259.37,475.0,387.55,NaN,-843.37,1039.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276334,2022-07-18 23:40,25.040,259.13,212.86,31.14,259.13,458.0,454.91,NaN,-1023.37,1073.46,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x = raw_data[['방류량']]

In [ ]:
x

,방류량
0,555.00
1,562.90
2,576.40
3,563.10
4,576.40
...,...
276331,259.23
276332,260.46
276333,259.37
276334,259.13


In [ ]:
# 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

input_x_regular = x.copy()

scaler = MinMaxScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
a = np.array(input_x_regular)
tmp_x = scaler.fit_transform(a.reshape(-1,1))
input_x_regular = pd.Series(list(tmp_x))

# 원하는 칼럼명 입력하기

In [ ]:
#ydata = raw_data[['현재수위','유입량','저수량','공용량','방류량','강화대교_조위','청담대교_유량','잠수교_유량','한강대교_유량']]
y = raw_data['방류량']

# 원하는 칼럼명 입력하기

In [ ]:
input_regular = pd.concat([input_x_regular, y], axis = 1)
input_regular

,0,방류량
0,[0.05768907641468924],555.00
1,[0.05798405615816888],562.90
2,[0.058488135466646746],576.40
3,[0.05799152399977596],563.10
4,[0.058488135466646746],576.40
...,...,...
276331,[0.04664525885405971],259.23
276332,[0.04669118607994324],260.46
276333,[0.04665048634318466],259.37
276334,[0.046641524933256163],259.13


# 원하는 칼럼명 입력하기

In [ ]:
nanIndex = raw_data[raw_data['방류량'].isnull() == True].index

In [ ]:
nanIndex

Int64Index([ 13200,  68081,  68083,  68084,  68085,  68086,  68087,  68089,
             68090,  68091,
            ...
            270533, 270534, 271588, 272215, 272216, 272217, 272218, 272219,
            272220, 272221],
           dtype='int64', length=743)

In [ ]:
from tqdm import tqdm

xdata = []
ydata = []
xdata_idx = []
ydata_idx = [-1]
for i in range(len(input_regular.columns)-1):
    xdata_idx.append(i)

w =  6 #window size

idx = []
for i in range(12):
  idx.append(i*26496)

for i in tqdm(range(11)):
    if i != 10:
        tmp = input_regular.iloc[idx[i]:idx[i+1], :]
    else:
        tmp = input_regular.iloc[idx[i]:, :]

    for j in range(len(tmp)-w):
        xdata.append(np.array(tmp.iloc[j:j + w, xdata_idx]).astype(float))

    for j in range(w,len(tmp)):
        ydata.append(np.array(tmp.iloc[j, ydata_idx]).astype(float))

100%|██████████| 11/11 [02:30<00:00, 13.64s/it]


In [ ]:
print(xdata_idx)
print(ydata_idx)

[0]
[-1]


In [ ]:
print(np.array(xdata).shape)
print(np.array(ydata).shape)

(276270, 6, 1)
(276270, 1)


In [ ]:
# 나중에 학습된 모델에 이거 넣어서 predict해서 결측치 채우기
x_real_predict = []
y_real_predict = []

for i in range(len(nanIndex)):
    k = nanIndex[-(i+1)]
    
    x_real_predict.insert(0,(xdata[k - w * (k // 26496 + 1)]))
    y_real_predict.insert(0,(ydata[k - w * (k // 26496 + 1)]))
    
    xdata.pop(k - w * (k // 26496 + 1))
    ydata.pop(k - w * (k // 26496 + 1))

In [ ]:
y_real_predict

[array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([nan]),
 array([na

In [ ]:
print(np.array(x_real_predict).shape)
print(np.array(y_real_predict).shape)

(743, 6, 1)
(743, 1)


In [ ]:
print(np.array(xdata).shape)
print(np.array(ydata).shape)

(275527, 6, 1)
(275527, 1)


In [ ]:
pd.DataFrame(ydata).isnull().sum()

0    0
dtype: int64

In [ ]:
xtrain = np.array(xdata[:-50000])
ytrain = np.array(ydata[:-50000])

xtest = np.array(xdata[-50000:])
ytest = np.array(ydata[-50000:])

In [ ]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(225527, 6, 1)
(225527, 1)
(50000, 6, 1)
(50000, 1)


In [ ]:
ytrain

array([[ 562. ],
       [ 563.1],
       [ 576.4],
       ...,
       [8235. ],
       [8235. ],
       [8241. ]])

## 모델 생성

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

if 'model_enc_dec_cnn' in locals(): 
    del model_enc_dec_cnn 
if 'history' in locals():
    del history

optimizer = tf.keras.optimizers.Adam(lr=0.01) #기본값 0.001

model_enc_dec_cnn = Sequential()
model_enc_dec_cnn.add(Conv1D(filters=32, kernel_size=2, padding = 'causal', activation='relu', input_shape=(w, xtrain.shape[2])))
#model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=2, padding = 'causal', activation='relu'))
#model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
model_enc_dec_cnn.add(Flatten())

model_enc_dec_cnn.add(RepeatVector(1))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(LSTM(200, activation='relu'))

model_enc_dec_cnn.add(Dense(100, activation='relu'))
model_enc_dec_cnn.add(Dense(1))

model_enc_dec_cnn.compile(loss=root_mean_squared_error, optimizer=optimizer, metrics =["accuracy"])

model_enc_dec_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 6, 32)             96        
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                                 
 repeat_vector (RepeatVector  (None, 1, 192)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 1, 200)            314400    
                                                                 
 lstm_1 (LSTM)               (None, 1, 200)            320800    
                                                                 
 lstm_2 (LSTM)               (None, 200)               320800    
                                                        

In [ ]:
# plot_model(model=model_enc_dec_cnn, show_shapes=True)

In [ ]:
epochs = 100
batch_size = 256
validation = 0.2

checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', save_weights_only=False, monitor='val_loss',
                                         mode='min', save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=10, mode='min')
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, mode='min', patience=3, min_lr=0.001)

In [ ]:
## 학습

history = model_enc_dec_cnn.fit(xtrain, ytrain, verbose=1, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])

Epoch 1/100
705/705 [==============================] - 39s 49ms/step - loss: nan - accuracy: 8.9790e-04 - val_loss: nan - val_accuracy: 0.0049 - lr: 0.0100
Epoch 2/100
705/705 [==============================] - 34s 48ms/step - loss: nan - accuracy: 8.9790e-04 - val_loss: nan - val_accuracy: 0.0049 - lr: 0.0100
Epoch 3/100
705/705 [==============================] - 34s 48ms/step - loss: nan - accuracy: 8.9790e-04 - val_loss: nan - val_accuracy: 0.0049 - lr: 0.0100
Epoch 4/100
705/705 [==============================] - 34s 48ms/step - loss: nan - accuracy: 8.9790e-04 - val_loss: nan - val_accuracy: 0.0049 - lr: 0.0050
Epoch 5/100
705/705 [==============================] - 34s 48ms/step - loss: nan - accuracy: 8.9790e-04 - val_loss: nan - val_accuracy: 0.0049 - lr: 0.0050
Epoch 6/100
705/705 [==============================] - 34s 48ms/step - loss: nan - accuracy: 8.9790e-04 - val_loss: nan - val_accuracy: 0.0049 - lr: 0.0050
Epoch 7/100
705/705 [==============================] - 34s 48ms/

In [ ]:
from sklearn import metrics

def scoring(y_true, y_pred):
  r2 = round(metrics.r2_score(y_true, y_pred)*100, 3)
  corr = round(np.corrcoef(y_true, y_pred)[0, 1], 3)
  mape = round(metrics.mean_absolute_error(y_true, y_pred, squared=False), 3)
  rmse = round(metrics.mean_squared_roor(y_true, y_pred, squared=False), 3)

  df = pd.DataFrame({
      'R2':r2,
      'Corr':corr,
      'RMSE':rmse,
      'mape':mape
  }, index = [0])
  return df

scoring(ytrain.y_true, ytrain.y_pred)

In [ ]:
## 평가

keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

yhat = keras_model_best.predict(xtest, verbose = 1)

yhat

keras_model_best.evaluate(xtest,ytest)

In [ ]:
## 학습 과정 시각화

fig, loss_ax = plt.subplots(1, 2)
fig.set_size_inches((15,5))

loss_ax[0].plot(history.history['loss'], 'b', label='train loss')
loss_ax[0].plot(history.history['val_loss'], 'r', label='val loss')

loss_ax[1].plot(history.history['accuracy'], 'y', label='train acc')
loss_ax[1].plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax[0].set_xlabel('epoch')
loss_ax[0].set_ylabel('loss')
loss_ax[1].set_xlabel('epoch')
loss_ax[1].set_ylabel('accuray')

loss_ax[0].legend(loc='upper right')
loss_ax[1].legend(loc='lower right')

plt.show()

In [ ]:
## 결과 시각화

tl_predict = np.array(yhat)

#결과 시각화
plt.figure(figsize=(15,10))
plt.plot(ytest, marker = ",", label = "actual")
plt.plot(tl_predict, marker = ",", label="predict", alpha = 0.5)
plt.legend(prop={'size':10})  

plt.show()

In [ ]:
## 점수 확인
print("RMSE : ",float(root_mean_squared_error(ytest, tl_predict)))

## 파일 저장

In [ ]:
x_real_predict = np.array(x_real_predict)
y_real_hat = keras_model_best.predict(x_real_predict, verbose = 1)

In [ ]:
for i in range(len(nanIndex)):
    raw_data['방류량'].iloc[nanIndex[i]] = np.float64(y_real_hat[i])

In [ ]:
raw_data['방류량'].to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/newdata7_방류량_단변량.csv', index = False, encoding = 'cp949')